<a href="https://colab.research.google.com/github/Marston/quant-apprentice/blob/main/the_quants_apprentice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quant Apprentice

⚙️ Agentic Workflow Patterns
Quant Apprentice implements three distinct workflow patterns that enable its complex, end-to-end analysis capabilities.

1. Prompt Chaining (Sequential Task Processing)

  This pattern is used for structured data processing tasks, particularly for news analysis. The agent executes a series of prompts in a specific order to transform raw data into a concise summary.

  Workflow: `Ingest News → Preprocess Text → Classify Sentiment → Extract Key Entities → Summarize Findings`

2. Routing (Specialized Sub-Agents)

  To handle diverse types of information, `the main agent routes tasks` to specialized sub-agents. Each sub-agent is an expert in a specific domain, ensuring that the right logic is applied to the right data.

  - Earnings Analyzer: Focuses on parsing quarterly earnings reports and financial statements.

  - News Analyzer: Specializes in sentiment analysis and summarization of market news.

  - Market Analyzer: Analyzes price action, volume, and technical indicators.

3. Evaluator–Optimizer Loop (Iterative Refinement)

  This is the agent's self-improvement mechanism. The system generates an initial analysis, evaluates it against a set of quality criteria, and then uses the feedback to generate a refined, more comprehensive final output.

  Workflow: `Generate Initial Analysis → Evaluate Quality & Completeness → Provide Feedback to Agent → Generate Refined Analysis`




In [ ]:
%pip install langgraph langchain langchain_google_genai yfinance

In [2]:
# Clones the project repository from GitHub
!git clone https://github.com/marston/Quant-Apprentice.git

Cloning into 'Quant-Apprentice'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [3]:
%cd Quant-Apprentice/

/content/Quant-Apprentice
